# 9 - Predições das matrículas

Para as matrículas, deseja-se atingir uma meta em relação à população. É possível prever como o número de matrículas vai crescer, mas a meta em relação à população precisa da estimativa da população para acontecer. 

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m')
dados_educ_pop = pd.read_csv('dados/dados_educ_pop.csv', sep = ',', parse_dates=['ANO_MES'], index_col='ANO_MES', date_parser=dateparse) 

# Só precisamos do distrito e da matrícula, para esse caso
dados_matricula = dados_educ_pop[['DISTRITO', 'MAT_CRECHE', 'DISTRITO_F']]

Usando o modelo de predição ARIMA, sigla para Média móvel integrada autoregressiva, que forneceu o menor erro médio nos testes, vamos prever como as matrículas vão se comportar. Além disso, vamos ver quando a meta de atendimento pode ser batida, baseando-se na média de população entre 1995 e 2018.

In [11]:
# Pegando os dados populacionais de 1995 a 2018
dados_pop = pd.read_csv('dados/populacao2.csv', sep = ';', encoding = 'ISO-8859-1')
dados_pop = dados_pop[["Localidades", "População de 0 a 3 Anos"]]
dados_pop.columns = ['DISTRITO', 'TOTAL']

# Calculando a média
dados_pop = dados_pop.groupby(['DISTRITO'])['TOTAL'].mean().to_frame().reset_index()

Agora, vamos gerar os modelos preditivos e comparar os resultados com a média populacional.

In [13]:
import statsmodels.api as sm

# Definindo a meta
meta = 0.8

for distrito in dados_matricula['DISTRITO'].unique():
    dados_matricula = dados_educ_pop[dados_educ_pop['DISTRITO'] == distrito]
    distrito_f = dados_matricula.at[dados_matricula.index[0], 'DISTRITO_F']

    # Gerando o modelo
    fit1 = sm.tsa.statespace.SARIMAX(dados_matricula['MAT_CRECHE'], order=(2, 1, 4),seasonal_order=(0,1,1,4)).fit()
    matricula_prevista = fit1.predict(start="2018-03-01", end="2030-12-01", dynamic=True)

    # Pegando a média populacional
    pop = dados_pop[dados_pop['DISTRITO'] == distrito_f]['TOTAL'].values[0]

    # Verificando se a meta foi atingida
    atingiu = False
    for index, element in matricula_prevista.iteritems():
        if element/pop >= meta:
            data = str(index)
            print(distrito_f + " - " + data[5:7] + '/' + data[:4])
            atingiu = True
            break
    if not atingiu:
            print("O distrito " + distrito_f + " só vai atingir a meta a partir de 2031.")

    # Gerando o gráfico
    plt.ioff()
    fig = plt.figure(dpi=200, figsize=(10, 6))
    plt.title('Evolução das matrículas das creches em ' + distrito)
    plt.plot(dados_matricula['MAT_CRECHE'], label='Matrículas registradas nas creches do distrito', color='c', linewidth=5.0)
    plt.plot(matricula_prevista, label='Matrículas previstas nas creches do distrito', color='midnightblue', linewidth=5.0)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2),  shadow=True, ncol=2)
    plt.ylabel('Crianças de 0 a 3 anos')
    plt.xlabel('Data de Referência')
    plt.xticks(rotation=90)
    plt.savefig('graficos/previsao_matricula_' + distrito + '.png', bbox_inches='tight')
    plt.close(fig)